# MCMC fault and flood estimation.

1. Sample from the posterior distribution f(P|D) to estimate parameter distributions.
2. Use MCMC samples to compute the flooding ensemble at each node for a specific design storm.

## Imports

In [37]:
# Library imports.
import emcee
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd


# Urbansurge imports.
from urbansurge import swmm_model
from urbansurge.fault_diagnosis.accuracy_metrics import true_positive_rate

# Autoreload.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model configurations

In [2]:
# SWMM model configuration file path.
config_path = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\lab_system_config.yml"

# Node where impulse is added.
input_node = 18

In [3]:
# Create instance of SWMM model.
swmm = swmm_model.SWMM(config_path)

# Configure model.
swmm.configure_model()

Simulation info
Flow Units: CFS
System Units: US
Start Time: 2020-01-01 00:00:00
Start Time: 2020-01-01 06:00:00


## Observation data

In [4]:
# Observation data filepath.
obs_fp = r"C:\Users\ay434\Documents\urbansurge\analysis\lab_system\diagnosis_pipeline\phy_impulse_fault_database.csv"

# Observation data frame.
obs_df = pd.read_csv(obs_fp)

### Select a fault scenario to diagnose

In [5]:
# Link ID for sensor.
link_id = 20

# Select a fault scenario to serve as the observed example. 
fault_component = 78
fault_value_idx = 20
fault_values = np.unique(obs_df.loc[obs_df['fault_component']==fault_component, 'fault_value'])
scenario = obs_df.loc[(obs_df['fault_component']==fault_component) & (obs_df['fault_value']==fault_values[fault_value_idx]), 'scenario'].iloc[0]

# Extract depth time series.
d_obs = obs_df.loc[obs_df['scenario']==scenario, f'Depth_link_{link_id}'].to_numpy()
d_obs += np.random.normal(0, 0.0, size=d_obs.shape)

# Extract velocity time series.
v_obs = obs_df.loc[obs_df['scenario']==scenario, f'Velocity_link_{link_id}'].to_numpy()
v_obs += np.random.normal(0, 0.0, size=d_obs.shape)

# Area of pipe.
area_obs = swmm.compute_area_from_depth(d_obs, link_id)

# Compute flow rate.
Q_obs = v_obs * area_obs

## MCMC of fault parameter space

In [6]:
# Fault parameters.
fault_params = {
    'fault_component': [],
    'fault_type': [],
    'lower_bound': [],
    'upper_bound': [],
}

#### Fault value type.
value_type = 'fraction'

#### Downstream links.
# All downstream links from input node.
downstream_links = swmm.get_downstream_components(input_node, 'Junction', 'Link')

#### Diameter fault.
# Find all links with the "clog" tag downstream of input node. Diameter fault.
clog_links = swmm.get_components_by_tag('clog')
clog_links = [int(l) for l in clog_links]
fault_components = np.intersect1d(downstream_links, clog_links)

# Add faults to parameter dictionary.
for fault_component in fault_components:
    fault_params['fault_type'].append('diameter') 
    fault_params['fault_component'].append(fault_component)
    fault_params['lower_bound'].append(0.1)
    fault_params['upper_bound'].append(0.9)

#### Roughness fault.
# Find all links downstream of input node that don't have "clog" tag. Roughness fault.
fault_components = [l for l in downstream_links if int(l) not in clog_links]

# Add faults to parameter dictionary.
for fault_component in fault_components:
    fault_params['fault_type'].append('roughness') 
    fault_params['fault_component'].append(fault_component)
    fault_params['lower_bound'].append(1)
    fault_params['upper_bound'].append(6)

#### Storage fault.
fault_components = [20]

# Add faults to parameter dictionary.
for fault_component in fault_components:
    fault_params['fault_type'].append('silting') 
    fault_params['fault_component'].append(fault_component)
    fault_params['lower_bound'].append(0.1)
    fault_params['upper_bound'].append(0.9)

#### Create fault scenario data frame.
fault_param_df = pd.DataFrame(fault_params)
fault_param_df.head()

,fault_component,fault_type,lower_bound,upper_bound
0,76,diameter,0.1,0.9
1,77,diameter,0.1,0.9
2,78,diameter,0.1,0.9
3,79,diameter,0.1,0.9
4,80,diameter,0.1,0.9


In [7]:
# Estimate the sigma of the uncertainty.
sigma = 0.01

In [ ]:
def log_prob(x, params):
    # Expected value of outcome is the model output with the fault parameters.
    for i, (param, param_name) in enumerate(zip(theta, parameter_names)):
        # Fault type, component, and value.
        fault_type, fault_component = param_name.split('_')
        fault_value = param # param is the parameter value.

        if fault_type == 'diameter':
            swmm = swmm_model.diameter_fault(swmm, fault_component, fault_value, value_type)

        elif fault_type == 'roughness':
            swmm = swmm_model.roughness_fault(swmm, fault_component, fault_value, value_type)

        elif fault_type == 'silting':
            swmm = swmm_model.silting_fault(swmm, fault_component, fault_value)

    # Run model.
    swmm.run_simulation()
    
    # Get the simulated flow.
    Q_est = swmm.get_link_flow(link_id)

    return np.reshape(Q_est, (-1, 1))

TypeError: itypes has to be a list of PyTensor types

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob, args=[means, cov])

In [28]:
basic_model = pm.Model()

with basic_model:
    # Priors for unknown parameters.
    pm_params = []
    parameter_names = []
    for i in range(fault_param_df.shape[0]):
        pname = '{}_{}'.format(fault_param_df.loc[i, 'fault_type'], fault_param_df.loc[i, 'fault_component'])
        parameter_names.append(pname)
        lower = fault_param_df.loc[i, 'lower_bound']
        upper = fault_param_df.loc[i, 'upper_bound']
        pm_params.append(pm.Uniform(pname, lower=lower, upper=upper))

    # Math stack the params.
    theta = pm.math.stack(pm_params)

    # # Observed likelihood is the sum of the squared differences between the observed and measured flow.
    # obs_like = np.sum(np.square(Q_obs - Q_est))
    Q_est = swmm_function(theta, parameter_names)

    # Likelihood (sampling distribution) of observations.
    Y_obs = pm.Normal("Y_obs", mu=Q_est, sigma=sigma, observed=np.reshape(Q_obs, (-1, 1)))

    # # Sampling with MCMC (Metropolis, NUTS, or other)
    # trace = pm.sample(3, return_inferencedata=False)
    


AttributeError: 'list' object has no attribute 'type'

In [35]:
print(pm_params[0].name)


diameter_76


In [ ]:
data = pd.DataFrame(dict(
    year = np.arange(1900., 1921., 1),
    lynx = np.array([4.0, 6.1, 9.8, 35.2, 59.4, 41.7, 19.0, 13.0, 8.3, 9.1, 7.4,
                8.0, 12.3, 19.5, 45.7, 51.1, 29.7, 15.8, 9.7, 10.1, 8.6]),
    hare = np.array([30.0, 47.2, 70.2, 77.4, 36.3, 20.6, 18.1, 21.4, 22.0, 25.4, 
                 27.1, 40.3, 57.0, 76.6, 52.3, 19.5, 11.2, 7.6, 14.6, 16.2, 24.7])))



[[0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.00280323]
 [0.10350483]
 [0.42012502]
 [0.59931051]
 [0.62159395]
 [0.58289583]
 [0.52471722]
 [0.46466755]
 [0.40693304]
 [0.35487988]
 [0.30943252]
 [0.269762  ]
 [0.23745371]
 [0.20864279]
 [0.18363095]
 [0.16259562]
 [0.14494112]
 [0.13012898]
 [0.11661169]
 [0.10477732]
 [0.0947484 ]
 [0.08596871]
 [0.07828513]
 [0.07155295]
 [0.0656631 ]
 [0.06052295]
 [0.0562107 ]
 [0.05230235]
 [0.04869076]
 [0.04538424]
 [0.04235434]
 [0.0395715 ]
 [0.03701193]
 [0.0346565 ]
 [0.03248935]
 [0.03049627]
 [0.02866589]
 [0.02698754]
 [0.02544821]
 [0.02405335]
 [0.02279613]
 [0.0216377 ]
 [0.0205699 ]
 [0.01958426]
 [0.01868124]
 [0.01